---
<center><h1>Fake News Prediction</h1></center>
<center><h3>Part of 30 Days 30 ML Projects Challenge</h3></center>

---

## 1) Understanding Problem Statement
---

**Fake news** (intentionally false information) spread through traditional media and online social networks is causing harm in society. It is on the rise, leading to deception and division. To combat this, we need a dependable system that can tell real news from fake news. Such a system is essential to rebuild trust in media and protect the truth in our online and offline world.

The goal of this project is to employ machine learning techniques **to classify news articles as either genuine or fake based on their content and characteristics**. This **classification task** is fundamental in addressing the challenge of fake news and promoting information integrity and informed society. Moreover, It also involves use of **Natural Language Processing (NLP) techniques** for handling textual data. 

## 2) Understanding Data
---
In this project, we work with a dataset referred to as **Fake News Data**. This dataset comprises various independent variables and one dependent variable for each individual news article.


### Dataset Description:

The dataset consists of news articles and the goal of this project is to utilize machine learning techniques to predict the reliability of these articles based on their content and associated attributes.

It includes the following attributes:

1. **id:** A unique identifier for each news article.
2. **title:** The title of the news article.
3. **author:** The author of the news article (if available).
4. **text:** The content of the article, which may be incomplete.
5. **label:** A categorical label indicating the potential reliability of the article:
   - 1: Denotes articles that are potentially unreliable or fake.
   - 0: Represents articles considered reliable or genuine.

## 3) Getting System Ready
---
Importing required libraries

In [2]:
import numpy as np
import pandas as pd

# for text data preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# for model buidling
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Downloading stop words for text preprocessing

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## 4) Data Eyeballing
---

### Laoding Data

In [5]:
fake_news_data = pd.read_csv('Datasets/Day4_Fake_News_Data.csv') 

In [6]:
fake_news_data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [8]:
print('The size of Dataframe is: ', fake_news_data.shape)
print('-'*100)
print('The Column Name, Record Count and Data Types are as follows: ')
fake_news_data.info()
print('-'*100)

The size of Dataframe is:  (20800, 5)
----------------------------------------------------------------------------------------------------
The Column Name, Record Count and Data Types are as follows: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB
----------------------------------------------------------------------------------------------------


In [9]:
# Defining numerical & categorical columns
numeric_features = [feature for feature in fake_news_data.columns if fake_news_data[feature].dtype != 'O']
categorical_features = [feature for feature in fake_news_data.columns if fake_news_data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 2 numerical features : ['id', 'label']

We have 3 categorical features : ['title', 'author', 'text']


In [10]:
print('Missing Value Presence in different columns of DataFrame are as follows : ')
print('-'*100)
total=fake_news_data.isnull().sum().sort_values(ascending=False)
percent=(fake_news_data.isnull().sum()/fake_news_data.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

Missing Value Presence in different columns of DataFrame are as follows : 
----------------------------------------------------------------------------------------------------


,Total,Percent
author,1957,9.408654
title,558,2.682692
text,39,0.187500
id,0,0.000000
label,0,0.000000


In [16]:
fake_news_data['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

## 5) Data Cleaning and Preprocessing
---

## 5) Model Building
---

### Creating Feature Matrix (Independent Variables) & Target Variable (Dependent Variable)

In [10]:
# separating the data and labels
X = diabetes_data.drop(columns = 'Outcome', axis=1) # Feature matrix
y = diabetes_data['Outcome'] # Target variable

In [11]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [12]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

### Data Standardization

In [13]:
scaler = StandardScaler()

In [14]:
scaler.fit(X)

StandardScaler()

In [15]:
standardized_data = scaler.transform(X)

In [16]:
standardized_data

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [17]:
X = standardized_data

In [18]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

### Train-Test Split

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=45)

In [20]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


In [21]:
print(y.shape, y_train.shape, y_test.shape)

(768,) (614,) (154,)


### Model Comparison : Training & Evaluation

In [22]:
models = [LogisticRegression, SVC, DecisionTreeClassifier, RandomForestClassifier]
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for model in models:
    classifier = model().fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

In [23]:
classification_metrics_df = pd.DataFrame({
    "Model": ["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],
    "Accuracy": accuracy_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "F1 Score": f1_scores
})

classification_metrics_df.set_index('Model', inplace=True)
classification_metrics_df

,Accuracy,Precision,Recall,F1 Score
Model,,,,
Logistic Regression,0.785714,0.769231,0.555556,0.645161
SVM,0.792208,0.789474,0.555556,0.652174
Decision Tree,0.675325,0.543478,0.462963,0.500000
Random Forest,0.766234,0.725000,0.537037,0.617021


### Inference
Best Model based on accuracy score only is Random Forest Classifier. However, for real life best model selection are not solely based on accuracy score, we need to take into account other evaluation metrics, business context and model interpretability.

The choice of the best model may depend on factors like the balance between precision and recall as well as overall accuracy. In this context, where the primary goal is to predict whether an individual has diabetes or not, the choice of the best model should prioritize predictive accuracy and reliability.

- Based on the provided metrics and considering the goal of accurately predicting diabetes, the **SVM (Support Vector Machine) model** appears to perform the best. It has the highest accuracy, precision and F1 score among the models you've evaluated.

- **Logistic Regression** and **Random Forest** also perform well and could be considered as alternatives.


**`Note:`** For real life best model selection are not solely based on accuracy score, we need to take into account other evaluation metrics, business context and model interpretability.